In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
from tqdm import tqdm
import json
import gc
import time
from datetime import timedelta

# STEP 3: Connecting Tariffed HS Codes, NAICS Codes and respective CUSMA Non-Utilisation Rates via Concordance Table

In [2]:
tariffed = pd.read_csv('tariff-hscodes.csv', encoding_errors='ignore', dtype={'HS Code': str})

tariffed['HS_Code_6digit'] = (
    tariffed['HS Code']
    .str.replace('.', '', regex=False)
    .str[:6]
)

tariffed = tariffed[['HS_Code_6digit', 'Category']].drop_duplicates()

In [3]:
concordance = pd.read_csv('C616_HS8toNaics6_concord_202505.csv', dtype={'hts10': str})

concordance['HS_Code_6digit'] = (
    concordance['HS8 Code']
    .astype(str)
    .str.zfill(8)
    .str[:6]
)

concordance['HS_Code_2digit'] = (
    concordance['HS8 Code']
    .astype(str)
    .str.zfill(8)
    .str[:2]
)

concordance['NAICS'] = concordance['NAICS 6 Code'].astype(str)

concordance = concordance[['HS_Code_6digit', 'NAICS', 'HS_Code_2digit']].drop_duplicates()

In [4]:
util = pd.read_csv('USMCA Utilization Data.csv')

util['HS_Code_2digit'] = (
    util['HS Classification']
    .astype(str)
    .str[:2]
)

util['nonutil_rate'] = util['USMCA_Nonutilisation_May2025']

util = util[['HS_Code_2digit', 'nonutil_rate']]

Setting the non-utilisation rate for those with sectoral tariffs at 1 reflects the fact that the 35% tariffs on non-CUSMA goods does not apply to the sectoral tariffs and that producers impacted by sectoral tariffs cannot use CUSMA to get their goods tariff-free.

In [5]:
naics_tar = concordance.merge(tariffed, on='HS_Code_6digit', how='left')

counts = naics_tar['HS_Code_6digit'].value_counts().reset_index()
naics_tarc = naics_tar.merge(counts, on='HS_Code_6digit', how='left')

naics_imp = naics_tarc.merge(util, on='HS_Code_2digit', how='left')

# Making the codes with a sectoral tariff category assigned to have a non-util rate value of 1
# This ensures that when multiplied later, sectoral tariffs do not affect the CUSMA non-utilisation rates to compute the impact of nonCUSMA 35% tariffs
naics_imp.loc[naics_imp['Category'].notna(), 'nonutil_rate'] = 1
naics_imp['Category'] = naics_imp['Category'].fillna('nonCUSMA')
naics_imp

,HS_Code_6digit,NAICS,HS_Code_2digit,Category,count,nonutil_rate
0,010110,112920,01,nonCUSMA,1,0.42
1,010121,112920,01,nonCUSMA,1,0.42
2,010129,112920,01,nonCUSMA,1,0.42
3,010130,112920,01,nonCUSMA,1,0.42
4,010190,112920,01,nonCUSMA,1,0.42
...,...,...,...,...,...,...
6969,961620,314990,96,nonCUSMA,1,0.84
6970,961700,332439,96,nonCUSMA,1,0.84
6971,961800,339990,96,nonCUSMA,1,0.84
6972,961900,322291,96,nonCUSMA,1,0.84


# STEP 4: Getting Weights by Province/Territory

Since multiple NAICS codes may contribute to the production of one HS code product, and we do not how much of a part does each NAICS contribute to the whole HS code good production, **thus an assumption is made to divide them equally**. Hence, when each export value is added, it is divided them by the count (how many times does that HS Code get repeated).  

Meanwhile, the non-utilisation rate of CUSMA exemption by each HS Code is first multiplied to the total value of each HS Code export to the US, before divided by the count.

In [6]:
# Initialize the DataFrame with NAICS data
tariff_exp_val = naics_imp.copy()

# Define all regions to process
provinces = ['NL', 'PEI', 'NS', 'NB', 'QC', 'ON', 'MB', 'SK', 'AL', 'BC', 'YK', 'NWT', 'NU']
cols = ['Commodity', 'Value ($)']

for province in provinces:
    # Process Global data
    global_df = pd.read_csv(f'{province}-Global.csv', usecols=cols)
    global_df['HS_Code_6digit'] = global_df['Commodity'].str.replace('.', '', regex=False).str[:6]
    global_df[f'{province}_Global'] = global_df['Value ($)']
    global_df = global_df[['HS_Code_6digit', f'{province}_Global']]
    
    tariff_exp_val = tariff_exp_val.merge(global_df, on='HS_Code_6digit', how='left')
    tariff_exp_val[f'{province}_Global'] = tariff_exp_val[f'{province}_Global'] / tariff_exp_val['count']
    
    # Process US data
    us_df = pd.read_csv(f'{province}-US.csv', usecols=cols)
    us_df['HS_Code_6digit'] = us_df['Commodity'].str.replace('.', '', regex=False).str[:6]
    us_df[f'{province}_US'] = us_df['Value ($)']
    us_df = us_df[['HS_Code_6digit', f'{province}_US']]
    
    tariff_exp_val = tariff_exp_val.merge(us_df, on='HS_Code_6digit', how='left')
    tariff_exp_val[f'{province}_US'] = (tariff_exp_val[f'{province}_US'] * tariff_exp_val['nonutil_rate']) / tariff_exp_val['count']

# Drop the non-relevant columns
tariff_exp_val = tariff_exp_val.drop(columns=['HS_Code_2digit', 'Category', 'count', 'nonutil_rate'])

tariff_exp_val = tariff_exp_val.fillna(0)

tariff_exp_val

,HS_Code_6digit,NAICS,NL_Global,NL_US,PEI_Global,PEI_US,NS_Global,NS_US,NB_Global,NB_US,...,AL_Global,AL_US,BC_Global,BC_US,YK_Global,YK_US,NWT_Global,NWT_US,NU_Global,NU_US
0,010110,112920,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,010121,112920,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,...,102006.0,42842.52,43608.0,18315.36,0.0,0.0,0.0,0.0,0.0,0.0
2,010129,112920,0.0,0.0,103341.0,43403.22,767570.0,322379.40,245243.0,103002.06,...,38107217.0,6846298.20,8940055.0,3722693.10,0.0,0.0,0.0,0.0,0.0,0.0
3,010130,112920,0.0,0.0,0.0,0.00,0.0,0.00,5591.0,2348.22,...,0.0,0.00,16453.0,6910.26,0.0,0.0,0.0,0.0,0.0,0.0
4,010190,112920,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,961620,314990,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,...,0.0,0.00,26374.0,19699.68,0.0,0.0,0.0,0.0,0.0,0.0
6970,961700,332439,0.0,0.0,0.0,0.00,9.0,0.00,0.0,0.00,...,4454.0,3741.36,191510.0,58298.52,0.0,0.0,0.0,0.0,0.0,0.0
6971,961800,339990,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,...,0.0,0.00,57388.0,30522.24,0.0,0.0,0.0,0.0,0.0,0.0
6972,961900,322291,0.0,0.0,0.0,0.00,0.0,0.00,113029441.0,94496776.08,...,10736.0,0.00,408726.0,4863.60,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Step 1: Aggregate ONLY the necessary sums (Global/US columns)
aggregates = {
    **{f'{province}_Global': (f'{province}_Global', 'sum') for province in provinces},
    **{f'{province}_US': (f'{province}_US', 'sum') for province in provinces},
}

weight_naics = tariff_exp_val.groupby('NAICS', as_index=False).agg(**aggregates)

# Step 2: Compute rates and keep ONLY those columns
for province in provinces:
    global_col = f'{province}_Global'
    us_col = f'{province}_US'
    rate_col = f'{province}'
    
    weight_naics[rate_col] = (
        weight_naics[us_col] / weight_naics[global_col]
    )

# Step 3: Select only naics, naics_2digit, and rate columns
final_columns = ['NAICS'] + [f'{province}' for province in provinces]
weight_naics = weight_naics[final_columns]
weight_naics = weight_naics.fillna(0)

# Result
weight_naics

,NAICS,NL,PEI,NS,NB,QC,ON,MB,SK,AL,BC,YK,NWT,NU
0,111110,0.000000,0.000000,0.000000,0.620000,0.016695,0.057076,0.064938,0.000728,0.024138,0.000000,0.00,0.0,0.00
1,111120,0.620000,0.620000,0.000000,0.124930,0.039934,0.371635,0.094353,0.047928,0.034204,0.028733,0.00,0.0,0.00
2,111130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00
3,111140,0.000000,0.000000,0.620000,0.620000,0.045639,0.032567,0.030494,0.038389,0.032376,0.028748,0.00,0.0,0.00
4,111150,0.000000,0.000000,0.000000,0.000000,0.194542,0.074227,0.370000,0.000000,0.370000,0.243865,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,339920,0.079019,0.227019,0.706480,0.243381,0.808154,0.653166,0.582863,0.866177,0.663320,0.587708,1.00,0.0,0.00
306,339930,0.358952,0.000000,0.397013,0.420000,0.347140,0.248730,0.337235,0.409485,0.368632,0.255254,0.00,0.0,0.42
307,339940,0.000000,0.000000,0.049719,0.831198,0.467514,0.732703,0.859608,0.150000,0.720095,0.691042,0.00,0.0,0.00
308,339950,0.000000,0.150000,0.149934,0.699527,0.748507,0.562703,0.151730,0.604759,0.206830,0.265313,0.15,0.0,0.15


# STEP 5: Using filtered NAICS Codes to filter directly-impacted businesses and estimate directly-impacted employees

In [8]:
auto_naics = set(naics_imp[naics_imp['Category'] == 'Auto']['NAICS'].unique())
alum_naics = set(naics_imp[naics_imp['Category'] == 'Aluminum']['NAICS'].unique())
steel_naics = set(naics_imp[naics_imp['Category'] == 'Steel']['NAICS'].unique())
copper_naics = set(naics_imp[naics_imp['Category'] == 'Copper']['NAICS'].unique())
lum_naics = set(naics_imp[naics_imp['Category'] == 'Lumber']['NAICS'].unique())
ene_naics = set(naics_imp[naics_imp['Category'] == 'Energy Mineral']['NAICS'].unique())
noncusma_naics = set(naics_imp[naics_imp['Category'] == 'nonCUSMA']['NAICS'].unique())
total_naics = set(naics_imp['NAICS'].unique())

In [9]:
col_i = ['DA',
        'All_Businesses', 'All_Employees',
        'Auto_B', 'Auto_E',
        'Alum_B', 'Alum_E',
        'Steel_B', 'Steel_E',
        'Cop_B', 'Cop_E',
        'Lum_B', 'Lum_E',
        'Ene_B', 'Ene_E',
        'CUSMA_B', 'CUSMA_E',
        'Total_B', 'Total_E',
        #add columns here, column titles are every value of total_naics
       ]

# Add individual NAICS codes as column headers for Est_Employees by NAICS
col_i.extend(sorted(total_naics))

business = pd.DataFrame(columns = col_i)

In [10]:
chunk_size = 1_000_000
province_code = {10:'NL',11:'PEI',12:'NS',13:'NB',24:'QC',35:'ON',46:'MB',47:'SK',48:'AL',59:'BC',60:'YK',61:'NWT',62:'NU'}

# melting weights in long form to prepare for a vectorized merge
wlong = (
    weight_naics
    .melt(id_vars='NAICS', var_name='Province', value_name='Rate')
)

# making a category boolean mask to prepare to include which NAICS business/employee value under which tariff category
def isin_mask(s, codes_set):
    # s is NAICS series
    return s.isin(codes_set) if len(codes_set) else pd.Series(False, index=s.index)

# loading necessary data
all_cols = pd.read_csv('Dec2022_Estabcounts_byDA.csv', encoding='ISO-8859-1', nrows=1).columns
cols_to_keep = [c for c in all_cols if c != 'Without employees']

# suggesting dtypes for performance purposes
dtype_hint = {
    '1-4':'Int64','5-9':'Int64','10-19':'Int64','20-49':'Int64',
    '50-99':'Int64','100-199':'Int64','200-499':'Int64','500 +':'Int64',
    'Total, with employees':'Int64',
}

total_start = time.time()
chunk_num = 0

# Preparing two empty lists
agg_frames = []       # List for weighted amount of businesses and est employees for each tariff
per_naics_frames = [] # List for total amount of employees for each NAICS code in each ADA --> needed for Step 8 later

for chunk in pd.read_csv(
        'Dec2022_Estabcounts_byDA.csv',
        encoding='ISO-8859-1',
        chunksize=chunk_size,
        usecols=cols_to_keep,
        dtype=dtype_hint,
    ):
    t0 = time.time()
    chunk_num += 1

    # 1) Filter non-relevant rows
    chunk = chunk[~chunk['NAICS'].isin(['Sub-total, classified', 'Unclassified', 'Total'])].copy()

    # 2) Basic transforms (vectorized)
    # keep NAICS 6-digit as string
    chunk['NAICS'] = chunk['NAICS'].astype(str).str[:6]
    chunk['Business_per_NAICS'] = chunk['Total, with employees'].fillna(0)

    # estimate employees (vectorized)
    cols = ['1-4','5-9','10-19','20-49','50-99','100-199','200-499','500 +']
    for c in cols:
        chunk['Est_Employees'] = (
            chunk['1-4'].fillna(0) * 3  +
            chunk['5-9'].fillna(0) * 7  +
            chunk['10-19'].fillna(0) * 15 +
            chunk['20-49'].fillna(0) * 35 +
            chunk['50-99'].fillna(0) * 75 +
            chunk['100-199'].fillna(0) * 150 +
            chunk['200-499'].fillna(0) * 350 +
            chunk['500 +'].fillna(0) * 550
        )

    # Province lookup
    # if 'DisseminationAre' isn’t numeric, ensure this still works (it uses first two chars)
    chunk['ProvinceCode'] = chunk['DisseminationAre'].astype(str).str[:2].astype(int, errors='ignore')
    chunk['Province'] = pd.Series(chunk['ProvinceCode']).map(province_code)

    # 3) Merge the per-(NAICS, Province) Rate (vectorized, no apply)
    merged = chunk.merge(wlong, how='left', on=['NAICS','Province'])

    # 4) Weighted columns (vectorized)
    merged['Weighted_Business']  = np.ceil(merged['Business_per_NAICS'] * merged['Rate'])
    merged['Weighted_Employees'] = np.ceil(merged['Est_Employees'] * merged['Rate'])

    # 5) Apply category masks to each of the tariff category to find the list of NAICS for each tariff type
    s = merged['NAICS']
    is_auto   = s.isin(auto_naics)
    is_alum   = s.isin(alum_naics)
    is_steel  = s.isin(steel_naics)
    is_copper = s.isin(copper_naics)
    is_lum    = s.isin(lum_naics)
    is_ene    = s.isin(ene_naics)
    is_cusma  = s.isin(noncusma_naics)
    is_total  = s.isin(total_naics)

    # 6) Build per-row contributions by multiplying masks (0/1) — vectorized
    wb = merged['Weighted_Business']
    we = merged['Weighted_Employees']

    merged['Auto_B']  = np.where(is_auto,   wb, 0)
    merged['Auto_E']  = np.where(is_auto,   we, 0)
    merged['Alum_B']  = np.where(is_alum,   wb, 0)
    merged['Alum_E']  = np.where(is_alum,   we, 0)
    merged['Steel_B'] = np.where(is_steel,  wb, 0)
    merged['Steel_E'] = np.where(is_steel,  we, 0)
    merged['Cop_B']   = np.where(is_copper, wb, 0)
    merged['Cop_E']   = np.where(is_copper, we, 0)
    merged['Lum_B']   = np.where(is_lum,    wb, 0)
    merged['Lum_E']   = np.where(is_lum,    we, 0)
    merged['Ene_B']   = np.where(is_ene,    wb, 0)
    merged['Ene_E']   = np.where(is_ene,    we, 0)
    merged['CUSMA_B'] = np.where(is_cusma,  wb, 0)
    merged['CUSMA_E'] = np.where(is_cusma,  we, 0)
    merged['Total_B'] = np.where(is_total,  wb, 0)
    merged['Total_E'] = np.where(is_total,  we, 0)

    # Always aggregate the unweighted totals too
    merged['All_Businesses'] = merged['Business_per_NAICS']
    merged['All_Employees']  = merged['Est_Employees']

    # 7) Chunk-level aggregation in ONE groupby
    agg_cols = [
        'All_Businesses','All_Employees',
        'Auto_B','Auto_E','Alum_B','Alum_E','Steel_B','Steel_E',
        'Cop_B','Cop_E','Lum_B','Lum_E','Ene_B','Ene_E',
        'CUSMA_B','CUSMA_E','Total_B','Total_E'
    ]
    by_da = merged.groupby('DisseminationAre', as_index=False)[agg_cols].sum()

    # 8) Generating the data for second list --> number of jobs per NAICS in each DA
    per_naics = (
        merged.loc[is_total, ['DisseminationAre','NAICS','Est_Employees']]
        .pivot_table(index='DisseminationAre', columns='NAICS', values='Est_Employees',
                     aggfunc='sum', fill_value=0)
        .reset_index()
    )

    # Saving the data into the two different lists in each chunk
    agg_frames.append(by_da)
    per_naics_frames.append(per_naics)

    print(f"Chunk {chunk_num} processed in {time.time()-t0:.2f} sec")

# Combine all chunks together to form one big dataframe
agg_all = pd.concat(agg_frames, ignore_index=True).groupby('DisseminationAre', as_index=False).sum()

per_naics_all = pd.concat(per_naics_frames, ignore_index=True).groupby('DisseminationAre', as_index=False).sum()

business = agg_all.merge(per_naics_all, on='DisseminationAre', how='left')

business = business.rename(columns={'DisseminationAre':'DA'})

print(f"\n✅ All chunks processed in {time.time()-total_start:.2f} sec")

business

Chunk 1 processed in 1.89 sec
Chunk 2 processed in 1.62 sec
Chunk 3 processed in 1.72 sec
Chunk 4 processed in 1.57 sec
Chunk 5 processed in 1.58 sec
Chunk 6 processed in 1.55 sec
Chunk 7 processed in 1.52 sec
Chunk 8 processed in 1.66 sec
Chunk 9 processed in 1.62 sec
Chunk 10 processed in 1.54 sec
Chunk 11 processed in 1.60 sec
Chunk 12 processed in 1.63 sec
Chunk 13 processed in 1.56 sec
Chunk 14 processed in 1.55 sec
Chunk 15 processed in 1.56 sec
Chunk 16 processed in 1.57 sec
Chunk 17 processed in 1.65 sec
Chunk 18 processed in 1.60 sec
Chunk 19 processed in 1.56 sec
Chunk 20 processed in 1.32 sec
Chunk 21 processed in 1.50 sec
Chunk 22 processed in 1.51 sec
Chunk 23 processed in 1.66 sec
Chunk 24 processed in 1.45 sec
Chunk 25 processed in 1.61 sec
Chunk 26 processed in 1.29 sec
Chunk 27 processed in 1.49 sec
Chunk 28 processed in 1.46 sec
Chunk 29 processed in 1.46 sec
Chunk 30 processed in 1.48 sec
Chunk 31 processed in 1.53 sec
Chunk 32 processed in 1.52 sec
Chunk 33 processe

,DA,All_Businesses,All_Employees,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,...,337215,337910,337920,339110,339910,339920,339930,339940,339950,339990
0,10000000,170,1006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10010165,22,266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10010166,2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10010167,6,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10010168,5,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55246,62080023,2,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55247,62080024,11,113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55248,62080025,11,356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55249,62080026,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# STEP 6: Regrouping filtered data into ADAs

In [11]:
da = gpd.read_file('lda_000b21a_e.shp')
da ['DA'] = da ['DAUID']
da ['DADGUID'] = da ['DGUID']
da = da[['DA', 'DADGUID']]

ada = gpd.read_file('lada000b21a_e.shp')
adac = ada.copy()
adac ['ADADGUID'] = adac ['DGUID']
adac = adac[['ADADGUID', 'geometry']]

relation = pd.read_csv('ada_da_relation.csv')

In [12]:
da_relation = da.merge(relation, on='DADGUID', how='left')
full_relation = da_relation.merge(adac, on='ADADGUID', how='left')
full_relation['DA'] = pd.to_numeric(full_relation['DA'], errors='coerce').astype('Int64')

In [13]:
business_merged =(
    business.merge(full_relation, on='DA', how='right')
    .fillna(0)
    .astype({col: 'int64' for col in business.columns if col != 'DA'})
) 

# Group by ADADGUID and aggregate
agg_dict = {
    'All_Businesses': ('All_Businesses', 'sum'),
    'All_Employees': ('All_Employees', 'sum'),
    'Auto_B': ('Auto_B', 'sum'),
    'Auto_E': ('Auto_E', 'sum'),
    'Alum_B': ('Alum_B', 'sum'),
    'Alum_E': ('Alum_E', 'sum'),
    'Steel_B': ('Steel_B', 'sum'),
    'Steel_E': ('Steel_E', 'sum'),
    'Cop_B': ('Cop_B', 'sum'),
    'Cop_E': ('Cop_E', 'sum'),
    'Lum_B': ('Lum_B', 'sum'),
    'Lum_E': ('Lum_E', 'sum'),
    'Ene_B': ('Ene_B', 'sum'),
    'Ene_E': ('Ene_E', 'sum'),
    'CUSMA_B': ('CUSMA_B', 'sum'),
    'CUSMA_E': ('CUSMA_E', 'sum'),
    'Total_B': ('Total_B', 'sum'),
    'Total_E': ('Total_E', 'sum'),
    'geometry': ('geometry', 'first')
}

# Add dynamic aggregation rules for each NAICS code
for naics in total_naics:
    agg_dict[naics] = (naics, 'sum')

# Perform grouped aggregation
business_grouped = business_merged.groupby('ADADGUID', as_index=False).agg(**agg_dict)

business_grouped

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\3982846873.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  business_grouped = business_merged.groupby('ADADGUID', as_index=False).agg(**agg_dict)


,ADADGUID,All_Businesses,All_Employees,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,...,311821,113311,212317,326196,312110,337127,333511,311340,311615,111910
0,2021S051610010001,219,2910,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2021S051610010002,66,394,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021S051610010003,288,4186,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2021S051610010004,438,9064,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2021S051610010005,203,1441,2,5,1,3,1,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,11,356,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5429,2021S051662080005,13,414,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5430,2021S051662080006,8,295,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# STEP 7: Processing it into Centroids for Counts and Choropleth for Rates

Since the earlier table shows the *weighted* numbers of directly exposed businesses and employees (by work location) together with *total* number of employees (by work location) for each affected NAICS code, the former is separated from the latter. The former needs to be processed into separate GDFs for centroids (to show counts) and choropleths (to show percentages)

In [14]:
excluded_cols = [
    'All_Businesses', 'All_Employees',
    'Auto_B', 'Auto_E', 'Alum_B', 'Alum_E',
    'Steel_B', 'Steel_E', 'Cop_B', 'Cop_E',
    'Lum_B', 'Lum_E', 'Ene_B', 'Ene_E', 
    'CUSMA_B', 'CUSMA_E', 'Total_B', 'Total_E', 
    'geometry'
]

business_filter = business_grouped[['ADADGUID'] + [col for col in business_grouped.columns if col in excluded_cols]]

business_census = business_grouped[[col for col in business_grouped.columns if col not in excluded_cols]]

In [15]:
# Convert to GeoDataFrame
cent_gdf = gpd.GeoDataFrame(business_filter, geometry='geometry', crs = 'EPSG:3347')

# Set a point within each polygon
cent_gdf = cent_gdf.drop(columns=['All_Businesses', 'All_Employees'])
cent_gdf['geometry'] = cent_gdf.geometry.representative_point()
cent_gdf.set_geometry('geometry', inplace=True)
cent_gdf

,ADADGUID,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,Cop_E,Lum_B,Lum_E,Ene_B,Ene_E,CUSMA_B,CUSMA_E,Total_B,Total_E,geometry
0,2021S051610010001,0,0,1,1,1,1,0,0,2,6,0,0,16,844,16,844,POINT (8927316.642 2156398.591)
1,2021S051610010002,0,0,0,0,0,0,0,0,0,0,0,0,2,10,2,10,POINT (8966678.602 2164982.461)
2,2021S051610010003,0,0,1,1,1,1,0,0,4,42,0,0,9,54,9,54,POINT (8934606.046 2144277.889)
3,2021S051610010004,0,0,1,1,1,1,1,1,0,0,2,12,7,55,7,55,POINT (8976138.327 2157799.904)
4,2021S051610010005,2,5,1,3,1,3,0,0,0,0,0,0,5,29,5,29,POINT (8970965.684 2157632.7)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (5259010.983 3653721.631)
5429,2021S051662080005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (6041286.124 3573109.546)
5430,2021S051662080006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (6281761.356 3557612.071)
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (5550011.303 3546406.473)


In [16]:
choro_cols = business_filter.copy()

tars = ['Auto', 'Alum', 'Steel', 'Cop', 'Lum', 'Ene', 'CUSMA', 'Total']

for tar in tars:
    tar_1 = f'{tar}_1'
    tar_2 = f'{tar}_2'

    choro_cols[tar_1] = (
        choro_cols[f'{tar}_B']/choro_cols['All_Businesses']
    )

    choro_cols[tar_2] = (
        choro_cols[f'{tar}_E']/choro_cols['All_Employees']
    )

choro_cols = choro_cols[['ADADGUID'] + [f'{tar}_1' for tar in tars] + [f'{tar}_2' for tar in tars] + ['geometry']]

choro_gdf = gpd.GeoDataFrame(choro_cols, geometry = 'geometry', crs = 'EPSG:3347')
choro_gdf

,ADADGUID,Auto_1,Alum_1,Steel_1,Cop_1,Lum_1,Ene_1,CUSMA_1,Total_1,Auto_2,Alum_2,Steel_2,Cop_2,Lum_2,Ene_2,CUSMA_2,Total_2,geometry
0,2021S051610010001,0.000000,0.004566,0.004566,0.000000,0.009132,0.000000,0.073059,0.073059,0.00000,0.000344,0.000344,0.00000,0.002062,0.000000,0.290034,0.290034,"MULTIPOLYGON (((8921559.157 2130255.903, 89215..."
1,2021S051610010002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025381,0.025381,"POLYGON ((8959571.943 2171799.486, 8959576.689..."
2,2021S051610010003,0.000000,0.003472,0.003472,0.000000,0.013889,0.000000,0.031250,0.031250,0.00000,0.000239,0.000239,0.00000,0.010033,0.000000,0.012900,0.012900,"MULTIPOLYGON (((8938088.457 2157739.16, 893808..."
3,2021S051610010004,0.000000,0.002283,0.002283,0.002283,0.000000,0.004566,0.015982,0.015982,0.00000,0.000110,0.000110,0.00011,0.000000,0.001324,0.006068,0.006068,"POLYGON ((8976008.48 2163749.357, 8976015.274 ..."
4,2021S051610010005,0.009852,0.004926,0.004926,0.000000,0.000000,0.000000,0.024631,0.024631,0.00347,0.002082,0.002082,0.00000,0.000000,0.000000,0.020125,0.020125,"POLYGON ((8969716.249 2163377.051, 8969785.883..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((5263454.031 3662224.177, 52634..."
5429,2021S051662080005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((6043125.089 3568426.163, 60431..."
5430,2021S051662080006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((6280121.289 3558103.571, 62801..."
5431,2021S051662080007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((5544792.883 3549525.897, 55447..."


# STEP 8: Generating Weight of Directly Exposed Jobs to Total Accessible Jobs in Each Industry

It is likely that while employees live close to their workplace, they do not live in the same ADA as they work in.  
  
StatsCan Census 2021 data shows a huge drop in the number of Canadians who travel more than 15km to their work vis-a-vis those who travel less than that distance to work.  
  
Thus, this cell creates a dictionary where for each ADA, it lists down, including itself, the ADA IDs within a 15km buffer around it (for small ADAs) or ADA IDs that are adjacent to it (for large ADAs). Small ADAs are defined as ADAs with an area less than (15km)^2.

In [17]:
# Copy from earlier ADA shapefile and ensure it's in projected CRS (EPSG:3347)
adas = ada.to_crs("EPSG:3347")

# Create a column to mark if ADA is "small" (<= 706 km2)
adas['is_small'] = adas['LANDAREA'] <= 706

# Build spatial index once
adas_sindex = adas.sindex

# Simplify geometries early to reduce memory use
adas['geometry'] = adas['geometry'].simplify(100)

# Prepare empty dictionary
ada_neighbors = {}

# Process in chunks to avoid RAM overload
chunk_size = 500
n = len(adas)

# Track overall time
start_time = time.time()

for start in tqdm(range(0, n, chunk_size)):
    chunk_start_time = time.time()
    end = min(start + chunk_size, n)
    chunk = adas.iloc[start:end].copy()

    for idx, row in chunk.iterrows():
        ada_uid = row['DGUID']
        geom = row['geometry']
        is_small = row['is_small']

        if is_small:
            buffer_geom = geom.buffer(15000)
            possible_matches_index = list(adas_sindex.intersection(buffer_geom.bounds))
            possible_matches = adas.iloc[possible_matches_index]
            matches = possible_matches[possible_matches.geometry.intersects(buffer_geom)]
        else:
            possible_matches_index = list(adas_sindex.intersection(geom.bounds))
            possible_matches = adas.iloc[possible_matches_index]
            matches = possible_matches[possible_matches.geometry.touches(geom) | (possible_matches['DGUID'] == ada_uid)]

        ada_neighbors[ada_uid] = matches['DGUID'].tolist()

    del chunk
    gc.collect()

    # Print chunk timing
    chunk_elapsed = time.time() - chunk_start_time
    print(f"Chunk {start}-{end} processed in {timedelta(seconds=chunk_elapsed)}")

# Overall timing
total_elapsed = time.time() - start_time
print(f"\nTotal time taken: {timedelta(seconds=total_elapsed)}")

# Optionally save the dictionary to disk
with open("ada_neighbors.json", "w") as f:
    json.dump(ada_neighbors, f)

  9%|███████▌                                                                           | 1/11 [00:33<05:33, 33.33s/it]

Chunk 0-500 processed in 0:00:33.331829


 18%|███████████████                                                                    | 2/11 [00:36<02:20, 15.59s/it]

Chunk 500-1000 processed in 0:00:03.165345


 27%|██████████████████████▋                                                            | 3/11 [00:40<01:21, 10.18s/it]

...

Chunk 4000-4500 processed in 0:00:02.035073


 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [02:46<00:11, 11.47s/it]

Chunk 4500-5000 processed in 0:00:08.623929


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [05:14<00:00, 28.63s/it]

Chunk 5000-5433 processed in 0:02:28.334659

Total time taken: 0:05:14.984763


The dictionary is then used in conjunction with the *total* count of employees (by work location), as separated in Cell 13 above, to find out the likely number of jobs of each 6-digit NAICS, and total number of jobs, that are 'accessible' from each ADA --> going by the assumption of travel distance made by Canadians to go to work from Census 2021 data

In [18]:
# Ensure 'ADADGUID' is the index for fast lookup
business_census_indexed = business_census.set_index('ADADGUID')

# Prepare list to collect results
aggregated_results = []

# Loop through ADA + its neighbors
for ada_id, neighbor_list in tqdm(ada_neighbors.items()):
    # Filter business_census rows for all neighbors
    rows = business_census_indexed.loc[business_census_indexed.index.intersection(neighbor_list)]
    
    # Sum across all rows (by column)
    summed = rows.sum()
    
    # Store result with ADA ID
    result = summed.to_dict()
    result['ADADGUID'] = ada_id
    
    aggregated_results.append(result)

# Convert to DataFrame
jobs = pd.DataFrame(aggregated_results)

100%|█████████████████████████████████████████████████████████████████████████████| 5433/5433 [00:07<00:00, 728.25it/s]


This is to find out the rate of directly exposed jobs (6-digit NAICS) to total jobs in each industry (1-digit NAICS jobs) that are accessible from each ADA

In [19]:
jobs_rate = jobs.copy()

cola = [col for col in jobs.columns if col != 'ADADGUID']

# Calculate summed groups by first digit of column name
jobs_rate['Sum1'] = jobs_rate[[col for col in cola if col.startswith('1')]].sum(axis=1)
jobs_rate['Sum2'] = jobs_rate[[col for col in cola if col.startswith('2')]].sum(axis=1)
jobs_rate['Sum3'] = jobs_rate[[col for col in cola if col.startswith('3')]].sum(axis=1)

# Compute share per column
for col in cola:
    col_rate = f'{col}_R'
    if col.startswith('1'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum1']
    elif col.startswith('2'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum2']
    elif col.startswith('3'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
    else:
        jobs_rate[col_rate] = 0  # fallback in case of unexpected prefix

# Final filtered DataFrame: only ADADGUID and the *_R columns
jobs_rate = jobs_rate[['ADADGUID'] + [f'{col}_R' for col in cola]]

jobs_rate = jobs_rate.fillna(0)

jobs_rate

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\3702249403.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\3702249403.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\3702249403.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

,ADADGUID,332329_R,334290_R,332431_R,331221_R,334220_R,336611_R,212397_R,331110_R,312140_R,...,311821_R,113311_R,212317_R,326196_R,312110_R,337127_R,333511_R,311340_R,311615_R,111910_R
0,2021S051610010001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,2021S051610010002,0.002472,0.0,0.0,0.0,0.0,0.123588,0.0,0.0,0.013418,...,0.026483,0.0,0.0,0.000000,0.067797,0.000000,0.0,0.0,0.124647,0.0
2,2021S051610010003,0.000000,0.0,0.0,0.0,0.0,0.004372,0.0,0.0,0.010201,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,2021S051610010004,0.001858,0.0,0.0,0.0,0.0,0.102176,0.0,0.0,0.010085,...,0.019904,0.0,0.0,0.019904,0.054936,0.001858,0.0,0.0,0.093684,0.0
4,2021S051610010005,0.001858,0.0,0.0,0.0,0.0,0.102176,0.0,0.0,0.010085,...,0.019904,0.0,0.0,0.019904,0.054936,0.001858,0.0,0.0,0.093684,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
5429,2021S051662080005,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
5430,2021S051662080006,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
5431,2021S051662080007,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


# STEP 9: Applying Jobs Weight to Census Data

Census 2021 Data reports residents' occupational NAICS code at the two-digit level

In [20]:
census = pd.read_csv('98-401-X2021012_English_CSV_data.csv', encoding='latin1')

census = census[census['CHARACTERISTIC_ID'].isin([2259, 2262, 2263, 2266])] # Only taking the relevant 2-digit NAICS codes from Census 2021 data

census['ADADGUID'] = census['DGUID']

census['ProvinceCode'] = census['ADADGUID'].str[9:11].astype(int)

census['Province'] = census['ProvinceCode'].map(province_code)

census['CHARACTERISTIC_NAME'] = (
    census['CHARACTERISTIC_NAME']
    .str.replace(' ', '', regex=False)
    .str[:2]
)

census = census[['ADADGUID', 'Province', 'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL']]

census_pivot = census.pivot_table(
    index=['ADADGUID', 'Province'],
    columns='CHARACTERISTIC_NAME',
    values='C1_COUNT_TOTAL',
    aggfunc='first'
).reset_index()

census_pivot

CHARACTERISTIC_NAME,ADADGUID,Province,11,21,31,To
0,2021S051610010001,NL,455.0,120.0,665.0,3715.0
1,2021S051610010002,NL,50.0,70.0,60.0,2120.0
2,2021S051610010003,NL,160.0,100.0,310.0,4530.0
3,2021S051610010004,NL,20.0,170.0,130.0,5145.0
4,2021S051610010005,NL,35.0,215.0,130.0,5190.0
...,...,...,...,...,...,...
4957,2021S051662080002,NU,10.0,15.0,10.0,860.0
4958,2021S051662080003,NU,0.0,0.0,0.0,210.0
4959,2021S051662080004,NU,10.0,0.0,0.0,510.0
4960,2021S051662080005,NU,10.0,10.0,0.0,465.0


Thus the weights from Step 8 is used to estimate how many employees (by primary residence) are working in industries directly exposed to tariffs

In [21]:
# Merge on ADADGUID to align both datasets
merged = census_pivot.merge(jobs_rate, on='ADADGUID', how='right')  # or 'left' if census is base

# Start building the adjusted DataFrame
adjusted_jobs = merged[['ADADGUID', 'Province', 'To']].copy()

# Compute adjusted values
for col in cola:
    rate_col = f'{col}_R'
    prefix = col[:2]

    if prefix in ['21', '22']:
        source_col = '21'
    elif prefix in ['31', '32', '33']:
        source_col = '31'
    else:
        source_col = prefix

    adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])

adjusted_jobs['Sum'] = adjusted_jobs.drop(columns=['ADADGUID', 'Province', 'To']).sum(axis=1)

adjusted_jobs = adjusted_jobs.fillna(0)

adjusted_jobs['Province'] = adjusted_jobs['Province'].mask(
    adjusted_jobs['Province'].isin([0, np.nan]),
    adjusted_jobs['ADADGUID'].str[9:11].astype(int).map(province_code)
)

adjusted_jobs

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_16980\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

,ADADGUID,Province,To,332329,334290,332431,331221,334220,336611,212397,...,113311,212317,326196,312110,337127,333511,311340,311615,111910,Sum
0,2021S051610010001,NL,3715.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1129.0
1,2021S051610010002,NL,2120.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8.0,0.0,225.0
2,2021S051610010003,NL,4530.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,590.0
3,2021S051610010004,NL,5145.0,1.0,0.0,0.0,0.0,0.0,14.0,0.0,...,0.0,0.0,3.0,8.0,1.0,0.0,0.0,13.0,0.0,379.0
4,2021S051610010005,NL,5190.0,1.0,0.0,0.0,0.0,0.0,14.0,0.0,...,0.0,0.0,3.0,8.0,1.0,0.0,0.0,13.0,0.0,437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,NU,510.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5429,2021S051662080005,NU,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5430,2021S051662080006,NU,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5431,2021S051662080007,NU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# STEP 10: Applying Export Weights to the Census Data

Export weights from Step 4 are applied to account for regional differences

In [22]:
# Step 1: Melt to long format
long_weight = weight_naics.melt(id_vars='NAICS', var_name='Province', value_name='Rate')

long_weight['NAICS'] = long_weight['NAICS'].astype(str) + '_r'

# Step 2: Pivot to wide format
weight_naics_pivot = long_weight.pivot(index=['Province'], columns='NAICS', values='Rate').reset_index()

weight_naics_pivot

NAICS,Province,111110_r,111120_r,111130_r,111140_r,111150_r,111160_r,111190_r,111211_r,111219_r,...,337215_r,337910_r,337920_r,339110_r,339910_r,339920_r,339930_r,339940_r,339950_r,339990_r
0,AL,0.024138,0.034204,0.0,0.032376,0.370000,0.000000,0.117974,0.0,0.026535,...,0.981173,0.514049,0.213624,0.532096,0.529703,0.663320,0.368632,0.720095,0.206830,0.425631
1,BC,0.000000,0.028733,0.0,0.028748,0.243865,0.000000,0.156439,0.0,0.000031,...,0.851043,0.186107,0.213990,0.590677,0.746073,0.587708,0.255254,0.691042,0.265313,0.269257
2,MB,0.064938,0.094353,0.0,0.030494,0.370000,0.000000,0.264963,0.0,0.127486,...,0.888928,0.790000,0.131253,0.450144,0.332359,0.582863,0.337235,0.859608,0.151730,0.573977
3,NB,0.620000,0.124930,0.0,0.620000,0.000000,0.000000,0.000000,0.0,0.004048,...,0.795159,0.790000,0.205144,0.849244,0.890000,0.243381,0.420000,0.831198,0.699527,0.330572
4,NL,0.000000,0.620000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.956908,0.000000,0.000000,0.416990,0.268019,0.079019,0.358952,0.000000,0.000000,0.132630
5,NS,0.000000,0.000000,0.0,0.620000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.905935,0.000000,0.000000,0.703223,0.786739,0.706480,0.397013,0.049719,0.149934,0.471560
6,NU,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.870000,0.890000,0.000000,0.420000,0.000000,0.150000,0.240000
7,NWT,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.890000,0.000000,0.000000,0.000000,0.000000,0.000000
8,ON,0.057076,0.371635,0.0,0.032567,0.074227,0.315716,0.206290,0.0,0.001356,...,0.948650,0.728146,0.187830,0.555835,0.907276,0.653166,0.248730,0.732703,0.562703,0.597974
9,PEI,0.000000,0.620000,0.0,0.000000,0.000000,0.000000,0.370000,0.0,0.000000,...,0.106944,0.000000,0.000000,0.000000,0.890000,0.227019,0.000000,0.000000,0.150000,0.485059


In [23]:
# Step 1: Merge adjusted_jobs with weight_naics_pivot on Province
census_byjobs = adjusted_jobs.merge(weight_naics_pivot, on='Province', how='left')

# Step 2: Multiply each column by its corresponding rate
for col in cola:
    rate_col = f'{col}_r'
    
    census_byjobs[col] = np.ceil(census_byjobs[col] * census_byjobs[rate_col])

# Step 3: Keep only ADADGUID and updated values
census_byjobs = census_byjobs[['ADADGUID', 'To'] + cola]

# Define output dictionary
grouped_data = {
    'ADADGUID': census_byjobs['ADADGUID'],  # retain ADA ID
    'Census': census_byjobs['To'],
    'Auto_C': census_byjobs[[col for col in cola if col in auto_naics]].sum(axis=1),
    'Alum_C': census_byjobs[[col for col in cola if col in alum_naics]].sum(axis=1),
    'Steel_C': census_byjobs[[col for col in cola if col in steel_naics]].sum(axis=1),
    'Cop_C': census_byjobs[[col for col in cola if col in copper_naics]].sum(axis=1),
    'Lum_C': census_byjobs[[col for col in cola if col in lum_naics]].sum(axis=1),
    'Ene_C': census_byjobs[[col for col in cola if col in ene_naics]].sum(axis=1),
    'CUSMA_C': census_byjobs[[col for col in cola if col in noncusma_naics]].sum(axis=1),
    'Total_C': census_byjobs.iloc[:, 2:].sum(axis=1)
}

# Create final grouped DataFrame
census_bytariffs = pd.DataFrame(grouped_data)
census_bytariffs

,ADADGUID,Census,Auto_C,Alum_C,Steel_C,Cop_C,Lum_C,Ene_C,CUSMA_C,Total_C
0,2021S051610010001,3715.0,0.0,1.0,1.0,0.0,5.0,0.0,613.0,613.0
1,2021S051610010002,2120.0,6.0,8.0,8.0,1.0,3.0,7.0,82.0,82.0
2,2021S051610010003,4530.0,0.0,1.0,2.0,0.0,8.0,28.0,258.0,258.0
3,2021S051610010004,5145.0,6.0,9.0,8.0,1.0,4.0,13.0,95.0,95.0
4,2021S051610010005,5190.0,6.0,9.0,8.0,1.0,4.0,15.0,107.0,107.0
...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,510.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5429,2021S051662080005,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5430,2021S051662080006,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5431,2021S051662080007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# STEP 11: Processing and adding the data to Choropleth and Centroid GDFs

Add the data on employees (by primary residence) to the GDFs produced in Step 7

In [24]:
centroids = cent_gdf.merge(census_bytariffs, on='ADADGUID', how='left')
centroids = centroids[['ADADGUID'] + [f'{tar}_B' for tar in tars] + [f'{tar}_E' for tar in tars] + [f'{tar}_C' for tar in tars] + ['geometry']]
centroids = centroids.to_crs('EPSG:4326')
centroids.to_file('centroids.geojson', driver='GeoJSON')
centroids

,ADADGUID,Auto_B,Alum_B,Steel_B,Cop_B,Lum_B,Ene_B,CUSMA_B,Total_B,Auto_E,...,Total_E,Auto_C,Alum_C,Steel_C,Cop_C,Lum_C,Ene_C,CUSMA_C,Total_C,geometry
0,2021S051610010001,0,1,1,0,2,0,16,16,0,...,844,0.0,1.0,1.0,0.0,5.0,0.0,613.0,613.0,POINT (-53.25419 47.86225)
1,2021S051610010002,0,0,0,0,0,0,2,2,0,...,10,6.0,8.0,8.0,1.0,3.0,7.0,82.0,82.0,POINT (-52.76054 47.72309)
2,2021S051610010003,0,1,1,0,4,0,9,9,0,...,54,0.0,1.0,2.0,0.0,8.0,28.0,258.0,258.0,POINT (-53.26649 47.73593)
3,2021S051610010004,0,1,1,1,0,2,7,7,0,...,55,6.0,9.0,8.0,1.0,4.0,13.0,95.0,95.0,POINT (-52.71312 47.62177)
4,2021S051610010005,2,1,1,0,0,0,5,5,5,...,29,6.0,9.0,8.0,1.0,4.0,15.0,107.0,107.0,POINT (-52.7703 47.64725)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-115.3713 67.80897)
5429,2021S051662080005,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-95.88322 68.64144)
5430,2021S051662080006,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-89.80822 68.53258)
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-107.82111 67.68532)


In [25]:
perc_bytariffs = census_bytariffs.copy()

for tar in tars:
    tar_3 = f'{tar}_3'

    perc_bytariffs[tar_3] = (
        perc_bytariffs[f'{tar}_C']/perc_bytariffs['Census']
    )

perc_bytariffs = perc_bytariffs[['ADADGUID'] + [f'{tar}_3' for tar in tars]]
perc_bytariffs['CUSMA_3'] = perc_bytariffs['CUSMA_3'].clip(upper=1)
perc_bytariffs['Total_3'] = perc_bytariffs['Total_3'].clip(upper=1)
perc_bytariffs

,ADADGUID,Auto_3,Alum_3,Steel_3,Cop_3,Lum_3,Ene_3,CUSMA_3,Total_3
0,2021S051610010001,0.000000,0.000269,0.000269,0.000000,0.001346,0.000000,0.165007,0.165007
1,2021S051610010002,0.002830,0.003774,0.003774,0.000472,0.001415,0.003302,0.038679,0.038679
2,2021S051610010003,0.000000,0.000221,0.000442,0.000000,0.001766,0.006181,0.056954,0.056954
3,2021S051610010004,0.001166,0.001749,0.001555,0.000194,0.000777,0.002527,0.018465,0.018465
4,2021S051610010005,0.001156,0.001734,0.001541,0.000193,0.000771,0.002890,0.020617,0.020617
...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5429,2021S051662080005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5430,2021S051662080006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5431,2021S051662080007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
choropleth = choro_gdf.merge(perc_bytariffs, on='ADADGUID', how='right')
choropleth = choropleth[['ADADGUID'] + [f'{tar}_1' for tar in tars] + [f'{tar}_2' for tar in tars] + [f'{tar}_3' for tar in tars] + ['geometry']]
choropleth = choropleth.to_crs('EPSG:4326')
choropleth

,ADADGUID,Auto_1,Alum_1,Steel_1,Cop_1,Lum_1,Ene_1,CUSMA_1,Total_1,Auto_2,...,Total_2,Auto_3,Alum_3,Steel_3,Cop_3,Lum_3,Ene_3,CUSMA_3,Total_3,geometry
0,2021S051610010001,0.000000,0.004566,0.004566,0.000000,0.009132,0.000000,0.073059,0.073059,0.00000,...,0.290034,0.000000,0.000269,0.000269,0.000000,0.001346,0.000000,0.165007,0.165007,"MULTIPOLYGON (((-53.51451 47.69912, -53.51464 ..."
1,2021S051610010002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.00000,...,0.025381,0.002830,0.003774,0.003774,0.000472,0.001415,0.003302,0.038679,0.038679,"POLYGON ((-52.78543 47.80961, -52.78542 47.809..."
2,2021S051610010003,0.000000,0.003472,0.003472,0.000000,0.013889,0.000000,0.031250,0.031250,0.00000,...,0.012900,0.000000,0.000221,0.000442,0.000000,0.001766,0.006181,0.056954,0.056954,"MULTIPOLYGON (((-53.12645 47.81702, -53.12663 ..."
3,2021S051610010004,0.000000,0.002283,0.002283,0.002283,0.000000,0.004566,0.015982,0.015982,0.00000,...,0.006068,0.001166,0.001749,0.001555,0.000194,0.000777,0.002527,0.018465,0.018465,"POLYGON ((-52.66902 47.66588, -52.66898 47.665..."
4,2021S051610010005,0.009852,0.004926,0.004926,0.000000,0.000000,0.000000,0.024631,0.024631,0.00347,...,0.020125,0.001156,0.001734,0.001541,0.000193,0.000771,0.002890,0.020617,0.020617,"POLYGON ((-52.73992 47.69568, -52.74026 47.694..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-115.34503 67.89695, -115.3453..."
5429,2021S051662080005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-95.82942 68.59941, -95.82932 ..."
5430,2021S051662080006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-89.84909 68.53759, -89.84931 ..."
5431,2021S051662080007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-107.9628 67.7012, -107.9628 6..."


In [27]:
choropleth.to_file('choropleth.geojson', driver='GeoJSON')
choropleth.to_file('choropleth.shp', driver='ESRI Shapefile')

# CSV Trails

for double-checking purposes

In [28]:
business_grouped.drop(columns='geometry').to_csv('trail.csv', index=False)
business_filter.drop(columns='geometry').to_csv('trail2.csv', index=False)
business_census.to_csv('trail3.csv', index=False)

In [29]:
choro_cols.drop(columns='geometry').to_csv('trail4.csv', index=False)

In [30]:
jobs.to_csv('trail5.csv')
jobs_rate.to_csv('trail6.csv')
adjusted_jobs.to_csv('trail7.csv')